In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="llama3.1:8b", temperature=0)

In [3]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

In [4]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model=model,
    tools=tools,
    checkpointer=InMemorySaver(),
    system_prompt="""You are a travel agent. No follow up questions. 
    Today's date is January 3, 2026. 
    IMPORTANT: All dates passed to tools MUST be in dd/mm/yyyy format and must be in the future."""
)

In [5]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Get me a direct flight from San Francisco to Tokyo on March 31st")]},
    config
    )

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Get me a direct flight from San Francisco to Tokyo on March 31st', additional_kwargs={}, response_metadata={}, id='fcc89d54-b829-46e8-ad81-20375dd3b962'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-01-03T21:17:29.535686839Z', 'done': True, 'done_reason': 'stop', 'total_duration': 186787604298, 'load_duration': 18356853585, 'prompt_eval_count': 1157, 'prompt_eval_duration': 155259965443, 'eval_count': 39, 'eval_duration': 13077066516, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019b85b6-1a95-7580-b6a1-eb935dac452c-0', tool_calls=[{'name': 'search-flight', 'args': {'departureDate': '31/03/2026', 'flyFrom': 'San Francisco', 'flyTo': 'Tokyo'}, 'id': '42081be2-7855-4aea-bbda-ab6d57d56e91', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1157, 'output_tokens': 39, 'total_tokens': 1196}),
              ToolMessage(content=[{'type'

In [7]:
print(response["messages"][-1].content)

This is a JSON array of objects, each representing a flight from San Francisco (SFO) to Tokyo (NRT). Here's a breakdown of the structure:

* Each object has several properties:
	+ `flyFrom`: The departure airport code (always "SFO" in this case).
	+ `flyTo`: The arrival airport code (always "NRT" in this case).
	+ `cityFrom` and `cityTo`: The city names corresponding to the departure and arrival airports.
	+ `departure` and `arrival`: Objects with two properties each:
		- `utc`: The date and time of departure/arrival in UTC format (e.g. "2026-04-01T00:43:00.000Z").
		- `local`: The date and time of departure/arrival in the local time zone (e.g. "2026-03-31T17:43:00.000").
	+ `totalDurationInSeconds` and `durationInSeconds`: The total duration of the flight in seconds.
	+ `price`: The price of the ticket.
	+ `deepLink`: A URL that can be used to book the flight.
	+ `currency`: The currency code (always "EUR" in this case).
* Each object also has a `layovers` property, which is an array 